In [1]:
pip install psycopg2-binary

     |████████████████████████████████| 3.4 MB 5.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys # import sys to get more detailed Python exception info
import psycopg2 # import the connect library for psycopg2
from psycopg2 import OperationalError, errorcodes, errors # import the error handling libraries for psycopg2
import psycopg2.extras as extras
import pandas as pd
import glob

In [3]:
path = r'/home/jovyan/work/market_history'
all_files = glob.glob(path + "/*")

li = []

for filename in all_files:
    print("Merging: ",filename)
    df = pd.read_csv(filename, index_col=None, header=0,sep=',')
    li.append(df)

marketData = pd.concat(li, axis=0, ignore_index=True)        

Merging:  /home/jovyan/work/market_history/Binance_ETCUSDT_d.csv
Merging:  /home/jovyan/work/market_history/Bitstamp_BCHUSD_d.csv
Merging:  /home/jovyan/work/market_history/Bitstamp_LTCUSD_d.csv


In [4]:
marketData.head()

,unix,date,symbol,open,high,low,close,Volume,Volume USDT,tradecount
0,1624665600000,2021-06-26 00:00:00,ETC/USDT,39.348,39.766,39.027,39.766,24260.184,9.546849e+05,2290.0
1,1624579200000,2021-06-25 00:00:00,ETC/USDT,43.156,45.499,38.946,39.350,2722988.803,1.143702e+08,331589.0
2,1624492800000,2021-06-24 00:00:00,ETC/USDT,40.769,43.558,38.726,43.151,2251950.724,9.289107e+07,185244.0
3,1624406400000,2021-06-23 00:00:00,ETC/USDT,36.536,42.978,34.550,40.763,3758417.658,1.477729e+08,244798.0
4,1624320000000,2021-06-22 00:00:00,ETC/USDT,39.452,42.800,32.210,36.545,5759701.293,2.135278e+08,321486.0


In [5]:
marketData['unix'] = marketData['unix'].apply(lambda x: pd.datetime.fromtimestamp(x) if x <= 1000000000000 else pd.datetime.fromtimestamp(x / 1000))
marketData['unix']

<ipython-input-5-dcb44580e54f>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  marketData['unix'] = marketData['unix'].apply(lambda x: pd.datetime.fromtimestamp(x) if x <= 1000000000000 else pd.datetime.fromtimestamp(x / 1000))


0      2021-06-26
1      2021-06-25
2      2021-06-24
3      2021-06-23
4      2021-06-22
          ...    
5734   2015-03-01
5735   2015-02-28
5736   2015-02-27
5737   2015-02-26
5738   2015-02-25
Name: unix, Length: 5739, dtype: datetime64[ns]

In [6]:
marketData['tradecount'].fillna(-1, inplace=True)

In [7]:
marketData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5739 entries, 0 to 5738
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   unix         5739 non-null   datetime64[ns]
 1   date         5739 non-null   object        
 2   symbol       5739 non-null   object        
 3   open         5739 non-null   float64       
 4   high         5739 non-null   float64       
 5   low          5739 non-null   float64       
 6   close        5739 non-null   float64       
 7   Volume       5739 non-null   float64       
 8   Volume USDT  5739 non-null   float64       
 9   tradecount   5739 non-null   float64       
dtypes: datetime64[ns](1), float64(7), object(2)
memory usage: 448.5+ KB


In [8]:
# Note: please change your database, username & password as per your own values
conn_params_dic = {
    "host"      : "postgres-source",
    "database"  : "db",
    "user"      : "root",
    "password"  : "toor"
}

In [9]:
# Define a function that handles and parses psycopg2 exceptions
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    
    # get the line number when exception occured
    line_n = traceback.tb_lineno    
    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type) 
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")
    
    
# Define a connect function for PostgreSQL database server
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successfully..................")
        
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)        
        # set the connection to 'None' in case of error
        conn = None
    return conn

def init_table(conn):
    cursor = conn.cursor()
    cursor.execute('''
DROP TABLE IF EXISTS marketdata ;

CREATE TABLE marketdata
(
    id SERIAL,
    ts TIMESTAMP WITHOUT TIME ZONE,
    symbol text,
    open double precision,
    high double precision,
    low double precision,
    close double precision,
    volume double precision,
    volumeUSDT double precision,  
    tradecount double precision,
    CONSTRAINT market_pkey PRIMARY KEY (id)
)''')
    

# Define function using copy_from_dataFile to insert the dataframe.
def copy_from_dataFile(conn, df, table):
    
#  Here we are going save the dataframe on disk as a csv file, load # the csv file and use copy_from() to copy it to the table
    tmp_df = "./df_temp.csv"
    df.to_csv(tmp_df, header=False,index = False)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",", columns=['ts', 'symbol', 'open', 'high', 'low', 'close', 'volume', 'volumeusdt', 'tradecount'])
        conn.commit()
        print("Data inserted using copy_from_datafile() successfully....")
        cursor.close()
    except (Exception, psycopg2.DatabaseError) as error:
        os.remove(tmp_df)
        # pass exception to function
        show_psycopg2_exception(err)
        cursor.close()
    os.remove(tmp_df)    

In [10]:
marketData.drop(['date'], axis=1, inplace=True)
marketData.rename(columns={"unix": "ts"}, inplace=True)
marketData.index.names = ['id']

In [11]:
marketData

,ts,symbol,open,high,low,close,Volume,Volume USDT,tradecount
id,,,,,,,,,
0,2021-06-26,ETC/USDT,39.348,39.766,39.027,39.766,24260.184,9.546849e+05,2290.0
1,2021-06-25,ETC/USDT,43.156,45.499,38.946,39.350,2722988.803,1.143702e+08,331589.0
2,2021-06-24,ETC/USDT,40.769,43.558,38.726,43.151,2251950.724,9.289107e+07,185244.0
3,2021-06-23,ETC/USDT,36.536,42.978,34.550,40.763,3758417.658,1.477729e+08,244798.0
4,2021-06-22,ETC/USDT,39.452,42.800,32.210,36.545,5759701.293,2.135278e+08,321486.0
...,...,...,...,...,...,...,...,...,...
5734,2015-03-01,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0
5735,2015-02-28,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0
5736,2015-02-27,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0


In [12]:
# Connect to the database
conn = connect(conn_params_dic)
conn.autocommit = True
#init table
init_table(conn)
# Run the copy_from_dataFile method, here saving data into 
copy_from_dataFile(conn, marketData, 'marketdata')
# Close the connection
#conn.close()

Connecting to the PostgreSQL...........
Connection successfully..................
Data inserted using copy_from_datafile() successfully....


In [13]:
# Prepare sql query
sql = "SELECT * FROM marketdata" 
cursor = conn.cursor()
# Execute cursor
cursor.execute(sql)
# Fetch all the records
tuples = cursor.fetchall()
# list of columns
cols =  list(marketData.columns)
#["id"] + print(cols)
marketdatadf = pd.DataFrame(tuples)
# Print few records
print(marketdatadf.head())

# Close the cursor
cursor.close()
# Close the connection
conn.close()

   0          1         2       3       4       5       6            7  \
0  1 2021-06-26  ETC/USDT  39.348  39.766  39.027  39.766    24260.184   
1  2 2021-06-25  ETC/USDT  43.156  45.499  38.946  39.350  2722988.803   
2  3 2021-06-24  ETC/USDT  40.769  43.558  38.726  43.151  2251950.724   
3  4 2021-06-23  ETC/USDT  36.536  42.978  34.550  40.763  3758417.658   
4  5 2021-06-22  ETC/USDT  39.452  42.800  32.210  36.545  5759701.293   

              8         9  
0  9.546849e+05    2290.0  
1  1.143702e+08  331589.0  
2  9.289107e+07  185244.0  
3  1.477729e+08  244798.0  
4  2.135278e+08  321486.0  


In [14]:
tuples

[(1,
  datetime.datetime(2021, 6, 26, 0, 0),
  'ETC/USDT',
  39.348,
  39.766,
  39.027,
  39.766,
  24260.184,
  954684.864643,
  2290.0),
 (2,
  datetime.datetime(2021, 6, 25, 0, 0),
  'ETC/USDT',
  43.156,
  45.499,
  38.946,
  39.35,
  2722988.803,
  114370221.064944,
  331589.0),
 (3,
  datetime.datetime(2021, 6, 24, 0, 0),
  'ETC/USDT',
  40.769,
  43.558,
  38.726,
  43.151,
  2251950.724,
  92891067.264792,
  185244.0),
 (4,
  datetime.datetime(2021, 6, 23, 0, 0),
  'ETC/USDT',
  36.536,
  42.978,
  34.55,
  40.763,
  3758417.658,
  147772942.84796,
  244798.0),
 (5,
  datetime.datetime(2021, 6, 22, 0, 0),
  'ETC/USDT',
  39.452,
  42.8,
  32.21,
  36.545,
  5759701.293,
  213527754.40704,
  321486.0),
 (6,
  datetime.datetime(2021, 6, 21, 0, 0),
  'ETC/USDT',
  50.949,
  51.44,
  38.507,
  39.461,
  4022053.682,
  177381507.654506,
  298789.0),
 (7,
  datetime.datetime(2021, 6, 20, 0, 0),
  'ETC/USDT',
  50.871,
  51.527,
  46.939,
  50.956,
  1347207.022,
  66532216.36933,
  

In [15]:
marketdatadf.set_index(0, inplace=True)

In [16]:
marketdatadf.rename(columns={0:'id',1:'ts',2:'symbol',3:'open',4:'high',5:'low',6:'close',7:'Volume',8:'Volume USDT',9:'tradecount'}
                    , inplace=True)
marketdatadf.index.names = ['id']

In [17]:
marketdatadf

,ts,symbol,open,high,low,close,Volume,Volume USDT,tradecount
id,,,,,,,,,
1,2021-06-26,ETC/USDT,39.348,39.766,39.027,39.766,24260.184,9.546849e+05,2290.0
2,2021-06-25,ETC/USDT,43.156,45.499,38.946,39.350,2722988.803,1.143702e+08,331589.0
3,2021-06-24,ETC/USDT,40.769,43.558,38.726,43.151,2251950.724,9.289107e+07,185244.0
4,2021-06-23,ETC/USDT,36.536,42.978,34.550,40.763,3758417.658,1.477729e+08,244798.0
5,2021-06-22,ETC/USDT,39.452,42.800,32.210,36.545,5759701.293,2.135278e+08,321486.0
...,...,...,...,...,...,...,...,...,...
5735,2015-03-01,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0
5736,2015-02-28,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0
5737,2015-02-27,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0


In [18]:
marketData

,ts,symbol,open,high,low,close,Volume,Volume USDT,tradecount
id,,,,,,,,,
0,2021-06-26,ETC/USDT,39.348,39.766,39.027,39.766,24260.184,9.546849e+05,2290.0
1,2021-06-25,ETC/USDT,43.156,45.499,38.946,39.350,2722988.803,1.143702e+08,331589.0
2,2021-06-24,ETC/USDT,40.769,43.558,38.726,43.151,2251950.724,9.289107e+07,185244.0
3,2021-06-23,ETC/USDT,36.536,42.978,34.550,40.763,3758417.658,1.477729e+08,244798.0
4,2021-06-22,ETC/USDT,39.452,42.800,32.210,36.545,5759701.293,2.135278e+08,321486.0
...,...,...,...,...,...,...,...,...,...
5734,2015-03-01,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0
5735,2015-02-28,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0
5736,2015-02-27,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0


In [19]:
marketdatadf

,ts,symbol,open,high,low,close,Volume,Volume USDT,tradecount
id,,,,,,,,,
1,2021-06-26,ETC/USDT,39.348,39.766,39.027,39.766,24260.184,9.546849e+05,2290.0
2,2021-06-25,ETC/USDT,43.156,45.499,38.946,39.350,2722988.803,1.143702e+08,331589.0
3,2021-06-24,ETC/USDT,40.769,43.558,38.726,43.151,2251950.724,9.289107e+07,185244.0
4,2021-06-23,ETC/USDT,36.536,42.978,34.550,40.763,3758417.658,1.477729e+08,244798.0
5,2021-06-22,ETC/USDT,39.452,42.800,32.210,36.545,5759701.293,2.135278e+08,321486.0
...,...,...,...,...,...,...,...,...,...
5735,2015-03-01,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0
5736,2015-02-28,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0
5737,2015-02-27,LTC/USDT,0.000,0.000,0.000,0.000,0.000,0.000000e+00,-1.0


In [20]:
marketdatadf['symbol'].unique()

array(['ETC/USDT', 'BCH/USDT', 'LTC/USDT'], dtype=object)